In [1]:
import pandas as pd
import math
import pickle
import datetime

# read in data

In [2]:
rows = []
with open('forumTest.data', "r") as fp:
    rows = [line.split(" ",1) for line in fp.readlines()]
test = pd.DataFrame(rows,columns=["subject","content"])
rows = []
with open('forumTraining.data', "r") as fp:
    rows = [line.split(" ",1) for line in fp.readlines()]
training = pd.DataFrame(rows,columns=["subject","content"])

# load

In [3]:
with open('vocab2.pickle','rb') as file:
    vocab2 = pickle.load(file)
with open('vocab3.pickle','rb') as file:
    vocab3 = pickle.load(file)
with open('trgroup.pickle','rb') as file:
    trgroup = pickle.load(file)
with open('probs.pickle','rb') as file:
    probs = pickle.load(file)
with open('text2.pickle','rb') as file:
    text2 = pickle.load(file)
with open('wtprob2.pickle','rb') as file:
    wtprob2 = pickle.load(file)
with open('text3.pickle','rb') as file:
    text3 = pickle.load(file)
with open('wtprob3.pickle','rb') as file:
    wtprob3 = pickle.load(file)

FileNotFoundError: [Errno 2] No such file or directory: 'vocab2.pickle'

# training

In [4]:
#collect the vocab
#run the pickle
vocab2 = {}
vocab3 = {}
for k,row in training.iterrows():
    words = row.content.split(" ")
    for i in range(len(words)-1):
        g2 = "%s %s"%(words[i],words[i+1])
        vocab2[g2] = 0
        try: #handles the edge case of the final index, which produces a 2-gram but is out of bounds for a 3-gram
            g3 = "%s %s %s"%(words[i],words[i+1],words[i+2])        
            vocab3[g3] = 0
        except IndexError:
            pass 
vocab2 = list(vocab2.keys())
vocab3 = list(vocab3.keys())
print('done')

done


# save 

In [ ]:
x = wtprob2
with open('vocab2.pickle','wb') as file:
    pickle.dump(vocab2,file)
with open('vocab3.pickle','wb') as file:
    pickle.dump(vocab3,file)
with open('trgroup.pickle','wb') as file:
    pickle.dump(trgroup,file)
with open('probs.pickle','wb') as file:
    pickle.dump(probs,file)
with open('text2.pickle','wb') as file:
    pickle.dump(text2,file)
with open('wtprob2.pickle','wb') as file:
    pickle.dump(wtprob2,file)
with open('text3.pickle','wb') as file:
    pickle.dump(text3,file)
with open('wtprob3.pickle','wb') as file:
    pickle.dump(wtprob3,file)

In [5]:
#run the pickle
#calculate probability estimates 
trgroup = training.groupby('subject')
docs ={}
probs = {g:0 for g in trgroup.groups.keys()}
text2 = {g:{} for g in trgroup.groups.keys()}
text3 = {g:{} for g in trgroup.groups.keys()}
wtprob2 = {g:{} for g in trgroup.groups.keys()} 
wtprob3 = {g:{} for g in trgroup.groups.keys()} 
for g, cont in trgroup:
    print(g+" "+str(datetime.datetime.now()))
    docs[g] = cont.content.tolist()
    probs[g] = len(cont)/len(training)
    print(g+" starting to split "+str(datetime.datetime.now()))
    for cnt in docs[g]:
        cnttxt = cnt.split(" ")
        for i in range(len(cnttxt)-1):
            try:
                text2[g]["%s %s"%(cnttxt[i],cnttxt[i+1])] += 1
            except KeyError:
                text2[g]["%s %s"%(cnttxt[i],cnttxt[i+1])] = 1
            try:
                try:
                    text3[g]["%s %s %s"%(cnttxt[i],cnttxt[i+1],cnttxt[i+2])] += 1
                except KeyError:
                    text3[g]["%s %s %s"%(cnttxt[i],cnttxt[i+1],cnttxt[i+2])] = 1 
            except IndexError:
                pass
    n2 = len(text2[g])
    n3 = len(text3[g])
    print(g+" starting 2grams "+str(datetime.datetime.now()))
    for gram in vocab2:
        nk = 0
        try:
            nk = text2[g][gram]
        except KeyError:
            pass
        wtprob2[g][gram] = math.log((nk + 1)/(n2+len(vocab2)))
    print(g+" starting 3grams"+str(datetime.datetime.now()))
    for gram in vocab3:
        nk = 0
        try:
            nk = text3[g][gram]
        except KeyError:
                pass
        wtprob3[g][gram] = math.log((nk + 1)/(n3+len(vocab3)))
print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

atheism 2019-02-17 19:00:34.477829
atheism starting to split 2019-02-17 19:00:34.478107
atheism starting 2grams 2019-02-17 19:00:34.942063
atheism starting 3grams2019-02-17 19:00:36.356791
autos 2019-02-17 19:00:39.185606
autos starting to split 2019-02-17 19:00:39.185895
autos starting 2grams 2019-02-17 19:00:39.536021
autos starting 3grams2019-02-17 19:00:40.750362
baseball 2019-02-17 19:00:43.745538
baseball starting to split 2019-02-17 19:00:43.745816
baseball starting 2grams 2019-02-17 19:00:44.143786
baseball starting 3grams2019-02-17 19:00:45.472160
christianity 2019-02-17 19:00:48.267390
christianity starting to split 2019-02-17 19:00:48.267655
christianity starting 2grams 2019-02-17 19:00:49.012381
christianity starting 3grams2019-02-17 19:00:50.287602
cryptology 2019-02-17 19:00:53.205385
cryptology starting to split 2019-02-17 19:00:53.205647
cryptology starting 2grams 2019-02-17 19:00:53.854385
cryptology starting 3grams2019-02-17 19:00:55.075016
electronics 2019-02-17 19:0

# Classify

In [6]:
win2 = 0.0
lose2 = 0.0
win3 = 0.0
lose3 = 0.0
for idx,tent in test.iterrows():
    maxnb2 = -9999999999
    maxg2 = ""
    maxnb3 = -9999999999
    maxg3 = ""
    if idx > 0 and idx%1000 == 0:
        print("%d win2:%d win3:%d %s"%(idx,win2,win3,str(datetime.datetime.now())))
    for g in trgroup.groups.keys():
        tot2 = math.log(probs[g])
        tot3 = math.log(probs[g])
        words = tent.content.split(" ")
        words2 = []
        words3 = []
        for i in range(len(words)-1):
            words2.append("%s %s"%(words[i],words[i+1]))
            try:
                words3.append("%s %s %s"%(words[i],words[i+1],words[i+2]))
            except IndexError:
                pass
        for word in words2:
            try:
                tot2 += wtprob2[g][word]     
            except KeyError:
                pass
        for word in words3:
            try:
                tot3 += wtprob3[g][word]     
            except KeyError:
                pass
        if tot2 > maxnb2:
            maxnb2 = tot2
            maxg2 = g
            
        if tot3 > maxnb3:
            maxnb3 = tot3
            maxg3 = g
    if maxg2 == tent.subject:
        win2 += 1.0
    else:
        lose2 += 1.0
    if maxg3 == tent.subject:
        win3 += 1.0
    else:
        lose3 += 1.0
print("2grams: %f%% correct ~ 3grams: %f%% correct "%(win2/len(test), win3/len(test)))

1000 win2:540 win3:553 2019-02-17 19:02:19.193523
2000 win2:1142 win3:1086 2019-02-17 19:02:29.113371
3000 win2:1843 win3:1786 2019-02-17 19:02:39.195139
4000 win2:2565 win3:2526 2019-02-17 19:02:50.114785
5000 win2:3291 win3:3253 2019-02-17 19:03:01.620427
6000 win2:3958 win3:3931 2019-02-17 19:03:16.662927
7000 win2:4868 win3:4782 2019-02-17 19:03:36.408110
2grams: 0.680659% correct ~ 3grams: 0.679065% correct 


In [8]:
topLst2 = [{'subject':'','word':'','score':0}]*30
topG2 = {g:[{'word':'','score':0}]*5 for g in trgroup.groups.keys()}
topLst3 = [{'subject':'','word':'','score':0}]*30
topG3 = {g:[{'word':'','score':0}]*5 for g in trgroup.groups.keys()}
for word in vocab2:
    cnt = False
    for w in word.split(' '):
        if(len(w)<2):
            cnt=True
    if(cnt):
        continue
    tot = 0
    cnt = 0
    for g in trgroup.groups.keys():
        try:
            tot += math.e**wtprob2[g][word]
            cnt += 1
        except KeyError as e:
            print(e)
            pass
    if cnt ==0:
        continue
    avg = tot/cnt
    for g in trgroup.groups.keys():
        try:
            v = (math.e**wtprob2[g][word])/avg
        except KeyError:
            print(e)
            continue
        for i in range(len(topG2[g])):
            if v > topG2[g][i]['score']:
                for j in range(len(topG2[g])-2,i,-1):
                    topG2[g][j] = topG2[g][j-1]
                topG2[g][i] = {'word':word,'score':v}
                break
        for i in range(len(topLst2)):
            if v > topLst2[i]['score']:
                for j in range(len(topLst2)-2,i,-1):
                    topLst2[j] = topLst2[j-1]
                topLst2[i] = {'subject':g,'word':word,'score':v}
                break
for word in vocab3:
    cnt = False
    for w in word.split(' '):
        if(len(w)<2):
            cnt=True
    if(cnt):
        continue
    tot = 0
    cnt = 0
    for g in trgroup.groups.keys():
        try:
            tot += math.e**wtprob3[g][word]
            cnt += 1
        except KeyError as e:
            print(e)
            pass
    if cnt ==0:
        continue
    avg = tot/cnt
    for g in trgroup.groups.keys():
        try:
            v = (math.e**wtprob3[g][word])/avg
        except KeyError:
            print(e)
            continue
        for i in range(len(topG3[g])):
            if v > topG3[g][i]['score']:
                for j in range(len(topG3[g])-2,i,-1):
                    topG3[g][j] = topG3[g][j-1]
                topG3[g][i] = {'word':word,'score':v}
                break
        for i in range(len(topLst3)):
            if v > topLst3[i]['score']:
                for j in range(len(topLst3)-2,i,-1):
                    topLst3[j] = topLst3[j-1]
                topLst3[i] = {'subject':g,'word':word,'score':v}
                break
for i in range(len(topLst2)-1,-1,-1):
    print("#%d subject:%s word:%s score:%f"%(i+1,topLst2[i]['subject'],topLst2[i]['word'],topLst2[i]['score']))
for i in range(len(topLst3)-1,-1,-1):
    print("#%d subject:%s word:%s score:%f"%(i+1,topLst3[i]['subject'],topLst3[i]['word'],topLst3[i]['score']))

orderG = list(trgroup.groups.keys())
orderG.sort()
for g in orderG:
    print(g+"~~~~~~~~~~~~~~~~~~~~~")
    for i in range(len(topG2[g])-1,-1,-1):
        print("\t%d word:%s score:%f"%(i+1,topG2[g][i]['word'],topG2[g][i]['score']))
    for i in range(len(topG3[g])-1,-1,-1):
        print("\t%d word:%s score:%f"%(i+1,topG3[g][i]['word'],topG3[g][i]['score']))

#30 subject:mideastpolitics word:policy research score:16.363078
#29 subject:space word:space station score:16.641208
#28 subject:hockey word:the flyers score:16.660889
#27 subject:space word:the moon score:16.726376
#26 subject:mideastpolitics word:the israeli score:16.838574
#25 subject:baseball word:the braves score:16.845635
#24 subject:xwindows word:output oname score:16.863796
#23 subject:medicine word:gordon banks score:16.903320
#22 subject:hockey word:the leafs score:16.929715
#21 subject:mideastpolitics word:serdar argic score:16.932812
#20 subject:mideastpolitics word:soviet armenia score:17.047479
#19 subject:politics word:ms myers score:17.056165
#18 subject:hockey word:the puck score:17.135718
#17 subject:mideastpolitics word:turks and score:17.281672
#16 subject:mideastpolitics word:the turks score:17.377391
#15 subject:cryptology word:public key score:17.448504
#14 subject:hockey word:power play score:17.460506
#13 subject:christianity word:apr athos score:17.512183
#12